In [ ]:
from manim import *
import numpy as np

# ------------------------------
# Helper Classes
# ------------------------------

class Arrow3D(Arrow):
    def __init__(self, start=ORIGIN, end=RIGHT, thickness=0.05, **kwargs):
        super().__init__(start, end, buff=0, stroke_width=2)
        self.set_start_and_end_attrs(start, end)

    def set_start_and_end_attrs(self, start, end):
        self.put_start_and_end_on(start, end)


class Line3D(Line):
    def __init__(self, start=ORIGIN, end=RIGHT, **kwargs):
        super().__init__(start, end, **kwargs)


class MatrixTransformHelper:
    @staticmethod
    def apply_matrix_to_vector(M, v):
        v = np.asarray(v).reshape(3,)
        return M.dot(v)


# ------------------------------
# Main Scene
# ------------------------------

class Covariance3D(ThreeDScene):
    def construct(self):
        # ---------------------- Parameters ----------------------
        mu = np.array([0.5, -0.2, 0.3])
        Sigma = np.array([
            [1.0, 0.6, 0.2],
            [0.6, 0.8, 0.1],
            [0.2, 0.1, 0.5],
        ])

        n_samples = 600
        short, medium, long = 0.8, 1.4, 2.2

        # ---------------------- Eigen Decomposition ----------------------
        eigvals, eigvecs = np.linalg.eigh(Sigma)
        order = np.argsort(eigvals)[::-1]
        eigvals = eigvals[order]
        eigvecs = eigvecs[:, order]
        S = eigvecs.dot(np.diag(np.sqrt(eigvals)))

        # ---------------------- Axes and Camera ----------------------
        axes = ThreeDAxes(x_range=[-3, 3, 1], y_range=[-3, 3, 1], z_range=[-3, 3, 1])
        self.add(axes)

        # Manual axis labels
        for direction, axis in zip([RIGHT, UP, OUT], ['x', 'y', 'z']):
            label = MathTex(axis, font_size=24, color=WHITE)
            label.next_to(3 * direction, direction)
            self.add_fixed_orientation_mobjects(label)

        self.set_camera_orientation(phi=60 * DEGREES, theta=-45 * DEGREES, distance=12)

        # ---------------------- Unit Cube + Transformed Cube ----------------------
        cube = Cube(side_length=1, fill_opacity=0.1, stroke_width=1)
        cube.move_to(ORIGIN)
        cube.scale(1.2)

        transformed_cube = cube.copy()

        def apply_linear_to_mobject(mobj, matrix, offset=np.zeros(3)):
            points = mobj.get_all_points()
            new_points = np.dot(points.reshape(-1, 3), matrix.T) + offset
            mobj.set_points_as_corners(new_points.reshape(points.shape))

        lbl_cube = Tex("Unit cube", font_size=24).next_to(cube, DOWN)
        lbl_tcube = Tex("Transformed cube", font_size=24).next_to(cube, DOWN).shift(RIGHT * 3)

        cube.to_edge(LEFT * 2)
        transformed_cube.to_edge(RIGHT * 2)
        transformed_cube_center = transformed_cube.get_center()
        transformed_cube.shift(-transformed_cube_center)
        apply_linear_to_mobject(transformed_cube, S)
        transformed_cube.shift(transformed_cube_center + mu)

        self.play(FadeIn(cube), Write(lbl_cube), run_time=short)
        self.play(FadeIn(transformed_cube), Write(lbl_tcube), run_time=short)
        self.wait(0.8)

        # ---------------------- Basis Transformation ----------------------
        basis = VGroup()
        basis_orig = VGroup()
        colors = [RED, GREEN, BLUE]
        for i, vec in enumerate([RIGHT, UP, OUT]):
            arr = Arrow3D(start=ORIGIN, end=vec * 1.5, thickness=0.03)
            arr.set_color(colors[i])
            basis.add(arr)

            tvec = MatrixTransformHelper.apply_matrix_to_vector(S, np.array(vec))
            arr_t = Arrow3D(start=mu, end=mu + tvec * 1.5, thickness=0.03)
            arr_t.set_color(colors[i])
            basis_orig.add(arr_t)

        b_labels = VGroup(*[MathTex(f"e_{i+1}") for i in range(3)])
        for i, label in enumerate(b_labels):
            label.next_to(basis[i].get_end(), UP * 0.3)

        self.play(LaggedStartMap(GrowArrow, basis, lag_ratio=0.2), run_time=medium)
        self.play(LaggedStartMap(FadeIn, b_labels, lag_ratio=0.15), run_time=short)
        self.wait(0.6)

        # Fixed Transform between basis and transformed basis
        self.play(
            LaggedStart(
                *[Transform(basis[i], basis_orig[i]) for i in range(len(basis))],
                lag_ratio=0.2
            ),
            run_time=long
        )
        self.wait(0.6)

        # ---------------------- Gaussian Cloud ----------------------
        rng = np.random.default_rng(seed=42)
        samples = rng.multivariate_normal(mean=mu, cov=Sigma, size=n_samples)

        dots = VGroup()
        for p in samples:
            s = Sphere(radius=0.03)
            s.move_to(p)
            s.set_fill(opacity=0.9)
            s.set_stroke(width=0.02)
            dots.add(s)

        self.play(LaggedStartMap(FadeIn, dots, lag_ratio=0.01), run_time=2.5)
        self.wait(0.6)

        # ---------------------- Covariance Ellipsoid ----------------------
        def ellipsoid(u, v):
            x = np.sin(u) * np.cos(v)
            y = np.sin(u) * np.sin(v)
            z = np.cos(u)
            point = np.dot(S, np.array([x, y, z])) + mu
            return point

        ellipsoid_surface = Surface(
            lambda u, v: ellipsoid(u, v),
            u_range=[0, PI],
            v_range=[0, TAU],
            resolution=(24, 24),
            fill_opacity=0.25,
            checkerboard_colors=[BLUE_E, BLUE_D],
        )

        self.play(FadeIn(ellipsoid_surface), run_time=1.6)
        self.wait(0.6)

        # ---------------------- Eigenvectors ----------------------
        eig_arrows = VGroup()
        eig_labels = VGroup()
        for i in range(3):
            vec = eigvecs[:, i]
            length = np.sqrt(eigvals[i]) * 1.6
            arr = Arrow3D(start=mu - vec * length, end=mu + vec * length, thickness=0.03)
            arr.set_color(colors[i])
            eig_arrows.add(arr)

            label = MathTex(f"\\lambda_{i+1}={eigvals[i]:.2f}")
            label.next_to(arr.get_end(), RIGHT * 0.2)
            eig_labels.add(label)

        self.play(LaggedStartMap(GrowArrow, eig_arrows, lag_ratio=0.2), run_time=short)
        self.play(LaggedStartMap(FadeIn, eig_labels, lag_ratio=0.15), run_time=short)
        self.wait(0.6)

        # ---------------------- Matrices on screen ----------------------
        sigma_tex = Matrix([[f"{Sigma[i,j]:.2f}" for j in range(3)] for i in range(3)])
        sigma_tex.scale(0.7)
        sigma_tex.to_corner(UL)

        decomp_tex = VGroup(
            MathTex("\\Sigma = U \\Lambda U^T", font_size=28),
            Matrix([[f"{eigvals[i]:.2f}" if i==j else "0.00" for j in range(3)] for i in range(3)]).scale(0.6),
        )
        decomp_tex.arrange(DOWN, buff=0.15)
        decomp_tex.next_to(sigma_tex, DOWN)

        self.play(Write(sigma_tex), run_time=short)
        self.play(FadeIn(decomp_tex), run_time=short)
        self.wait(0.8)

        # ---------------------- Camera Motion ----------------------
        self.begin_ambient_camera_rotation(rate=0.12)
        self.play(self.camera.frame.animate.scale(0.9), run_time=3)
        self.wait(2)
        self.stop_ambient_camera_rotation()

        self.play(self.camera.frame.animate.move_to(ellipsoid_surface.get_center()).scale(0.6), run_time=1.6)
        self.wait(1.2)

        # ---------------------- Legend ----------------------
        legend = VGroup(
            MathTex("\\text{Cloud (samples)}"),
            MathTex("\\text{Ellipsoid (covariance)}"),
            MathTex("\\text{Eigenvectors / principal axes}"),
        )
        legend.arrange(DOWN, aligned_edge=LEFT)
        legend.scale(0.6)
        legend.to_corner(DL)

        self.play(FadeIn(legend), run_time=short)
        self.wait(2)
        self.play(FadeOut(legend), FadeOut(sigma_tex), FadeOut(decomp_tex), FadeOut(lbl_cube), FadeOut(lbl_tcube))
        self.wait(0.8)


%manim -ql -v warning Covariance3D

UsageError: Cell magic `%%manim` not found.
